In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from sklearn.model_selection import StratifiedKFold

In [2]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [3]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [5]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [10]:
from tensorflow.python.keras.callbacks import EarlyStopping

X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
cv_scores = []

for train_index, test_index in skf.split(X_data, y_data):
    print(f"\nTraining Fold {fold}...")

    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    model = model_builder.build_conv_pool_model(input_size, len(df_names))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train,
                        epochs=20,
                        batch_size=32,
                        callbacks=[early_stopping],
                        validation_data=(X_test, y_test),
                        verbose=0)

    for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
        print(f"  Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

    score = model.evaluate(X_test, y_test, verbose=0)
    print(f"Fold {fold} Final Accuracy: {score[1] * 100:.2f}%")
    cv_scores.append(score[1])
    fold += 1


Training Fold 1...
  Epoch 1: val_loss=0.5426, val_accuracy=0.8335
  Epoch 2: val_loss=0.4405, val_accuracy=0.8521
  Epoch 3: val_loss=0.3907, val_accuracy=0.8638
  Epoch 4: val_loss=0.3454, val_accuracy=0.8782
  Epoch 5: val_loss=0.3145, val_accuracy=0.8841
  Epoch 6: val_loss=0.3145, val_accuracy=0.8866
  Epoch 7: val_loss=0.3242, val_accuracy=0.8820
  Epoch 8: val_loss=0.2967, val_accuracy=0.8946
  Epoch 9: val_loss=0.2791, val_accuracy=0.8998
  Epoch 10: val_loss=0.3431, val_accuracy=0.8818
  Epoch 11: val_loss=0.2629, val_accuracy=0.9051
  Epoch 12: val_loss=0.2663, val_accuracy=0.9036
  Epoch 13: val_loss=0.2567, val_accuracy=0.9073
  Epoch 14: val_loss=0.2419, val_accuracy=0.9098
  Epoch 15: val_loss=0.2689, val_accuracy=0.9058
  Epoch 16: val_loss=0.2468, val_accuracy=0.9103
  Epoch 17: val_loss=0.2421, val_accuracy=0.9110
  Epoch 18: val_loss=0.2375, val_accuracy=0.9132
  Epoch 19: val_loss=0.2641, val_accuracy=0.9066
  Epoch 20: val_loss=0.2474, val_accuracy=0.9105
Fold 1 Fi

In [11]:
print("\nCross-Validation Results:")
print(f"Mean Accuracy: {np.mean(cv_scores) * 100:.2f}%")
print(f"Standard Deviation: {np.std(cv_scores) * 100:.2f}%")


Cross-Validation Results:
Mean Accuracy: 90.89%
Standard Deviation: 0.14%


In [7]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [8]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 4ms/step


In [12]:
model.save("models/model_9089")

INFO:tensorflow:Assets written to: models/model_9089/assets


INFO:tensorflow:Assets written to: models/model_9089/assets


In [11]:
filtered_df = test_df[test_df['pred'] == '9']
filtered_df[['label', 'pred']]

,label,pred
33,Test4779.png,9
82,Test5117.png,9
123,Test5739.png,9
159,Test6581.png,9
173,Test4157.png,9
...,...,...
6829,Test5906.png,9
6835,Test5523.png,9
6869,Test5286.png,9
6973,Test5052.png,9
